<a href="https://colab.research.google.com/github/cooeoeooc/2178_g38/blob/main/group38_code_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import numpy as np

# Data Preprocessing and Cleaning

In [38]:
data = pd.read_csv("https://raw.githubusercontent.com/cooeoeooc/2178_g38/main/Arrests_and_Strip_Searches_(RBDC-ARR-TBL-001).csv")
data.head(5)

,Arrest_Year,Arrest_Month,EventID,ArrestID,PersonID,Perceived_Race,Sex,Age_group__at_arrest_,Youth_at_arrest__under_18_years,ArrestLocDiv,...,Actions_at_arrest___Resisted__d,Actions_at_arrest___Mental_inst,Actions_at_arrest___Assaulted_o,Actions_at_arrest___Cooperative,SearchReason_CauseInjury,SearchReason_AssistEscape,SearchReason_PossessWeapons,SearchReason_PossessEvidence,ItemsFound,ObjectId
0,2020,July-Sept,1005907,6017884.0,326622,White,M,Aged 35 to 44 years,Not a youth,54,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,1
1,2020,July-Sept,1014562,6056669.0,326622,White,M,Aged 35 to 44 years,Not a youth,54,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2
2,2020,Oct-Dec,1029922,6057065.0,326622,Unknown or Legacy,M,Aged 35 to 44 years,Not a youth,54,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,3
3,2021,Jan-Mar,1052190,6029059.0,327535,Black,M,Aged 25 to 34 years,Not a youth,XX,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4
4,2021,Jan-Mar,1015512,6040372.0,327535,South Asian,M,Aged 25 to 34 years,Not a youth,XX,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,5


In [39]:
def test_na_num(df):
    for column in df:
        print(sum(df[column].isna()), column)

In [40]:
test_na_num(data)

0 Arrest_Year
0 Arrest_Month
0 EventID
469 ArrestID
0 PersonID
4 Perceived_Race
0 Sex
24 Age_group__at_arrest_
0 Youth_at_arrest__under_18_years
0 ArrestLocDiv
0 StripSearch
0 Booked
165 Occurrence_Category
0 Actions_at_arrest___Concealed_i
0 Actions_at_arrest___Combative__
0 Actions_at_arrest___Resisted__d
0 Actions_at_arrest___Mental_inst
0 Actions_at_arrest___Assaulted_o
0 Actions_at_arrest___Cooperative
57475 SearchReason_CauseInjury
57475 SearchReason_AssistEscape
57475 SearchReason_PossessWeapons
57475 SearchReason_PossessEvidence
57475 ItemsFound
0 ObjectId


In [41]:
data = data.dropna(subset = ["ArrestID", "Perceived_Race", "Age_group__at_arrest_", "Occurrence_Category"])

Cleaning existing variables. 

In [46]:
data["Age_group_clean"] = data["Age_group__at_arrest_"].replace({"Aged 17 years and younger": "Aged 17 years and under", "Aged 65 and older": "Aged 65 years and older"})

Establishing exclusion criteria – based on popular Occurrence Categories. 

In [47]:
data["Occurrence_Category"].value_counts() # seeing most popular categories

occurrence_replacement_dict = {"Assault": 0, 
                              "Assault & Other crimes against persons": 1, 
                              "Robbery & Theft": 2, 
                              "Robbery/Theft": 2, 
                              "Warrant": 3, 
                              "FTA/FTC/Compliance Check/Parollee": 4, 
                              "FTA/FTC, Compliance Check & Parollee": 4, 
                              "Police Category - Administrative": 5, 
                              "Drug Related": 6} # should we add more categories?

data["Occurrence_Category"] = data["Occurrence_Category"].replace(occurrence_replacement_dict)
data["Occurrence_Category"] = data["Occurrence_Category"].where(data["Occurrence_Category"].isin(list(occurrence_replacement_dict.values())) == True, 7)

Creating new variables. 

In [48]:
data["SearchReasonCount"] = data["SearchReason_CauseInjury"] + data["SearchReason_AssistEscape"] + data["SearchReason_PossessWeapons"] + data["SearchReason_PossessEvidence"]

In [49]:
data["UncooperativeActionsCount"] = data["Actions_at_arrest___Concealed_i"] + data["Actions_at_arrest___Combative__"] + data["Actions_at_arrest___Resisted__d"] + data["Actions_at_arrest___Mental_inst"] + data["Actions_at_arrest___Assaulted_o"]

In [50]:
data["Time_series"] = np.where((data["Arrest_Year"] == 2020) & (data["Arrest_Month"] == "Jan-Mar"), 1, 
                               (np.where((data["Arrest_Year"] == 2020) & (data["Arrest_Month"] == "Apr-June"), 2, 
                                         (np.where((data["Arrest_Year"] == 2020) & (data["Arrest_Month"] == "July-Sept"), 3, 
                                                   (np.where((data["Arrest_Year"] == 2020) & (data["Arrest_Month"] == "Oct-Dec"), 4, 
                                                             (np.where((data["Arrest_Year"] == 2021) & (data["Arrest_Month"] == "Jan-Mar"), 5, 
                                                                       (np.where((data["Arrest_Year"] == 2021) & (data["Arrest_Month"] == "Apr-June"), 6, 
                                                                                 (np.where((data["Arrest_Year"] == 2021) & (data["Arrest_Month"] == "July-Sept"), 7, 
                                                                                           (np.where((data["Arrest_Year"] == 2021) & (data["Arrest_Month"] == "Oct-Dec"), 8, 0)))))))))))))))

# EDA

## Visualizations

## t-tests

# Feature Engineering

## Numerical representation

In [52]:
race_dict = {"White": 0,
             "Unknown or Legacy": 1,
             "Black":2,
             "South Asian":3,
             "Indigenous":4,
             "Middle-Eastern":5, 
             "Latino": 6,
             "East/Southeast Asian":7}

data['Perceived_Race'] = data['Perceived_Race'].replace(race_dict)

In [53]:
sex_dict= {'M':0,'F':1,'U':2}

data['Sex'] = data['Sex'].replace(sex_dict)

In [54]:
age_dict={'Aged 17 years and under': 0,
          'Aged 18 to 24 years': 1,
          'Aged 25 to 34 years': 2,
          'Aged 35 to 44 years': 3,
          'Aged 45 to 54 years': 4,
          'Aged 55 to 64 years': 5,
          'Aged 65 years and older': 6,
          }

data['Age_group_clean'] = data['Age_group_clean'].replace(age_dict)

In [55]:
youth_dict={'Youth (aged 17 and younger)': 0,
            'Youth (aged 17 years and under)': 0, 
            'Not a youth': 1}

data['Youth_at_arrest__under_18_years'] = data['Youth_at_arrest__under_18_years'].replace(youth_dict)

In [56]:
data = data.dropna()

## One-hot encoding

# Power Analysis